In [3]:
import numpy as np
import pandas as pd
import random
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import 

In [4]:
trainData = pd.read_csv("train data.csv")
testData = pd.read_csv("test data.csv")

In [5]:
trainData

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,...,ROP,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE
0,15-09-2013,494.528,437641.9688,6470972.5,-469.501831,NORDLAND GP.,NaN,19.480835,NaN,1.611410,...,34.636410,NaN,NaN,-0.574928,NaN,NaN,NaN,NaN,65000,1.0
1,15-09-2013,494.680,437641.9688,6470972.5,-469.653809,NORDLAND GP.,NaN,19.468800,NaN,1.618070,...,34.636410,NaN,NaN,-0.570188,NaN,NaN,NaN,NaN,65000,1.0
2,15-09-2013,494.832,437641.9688,6470972.5,-469.805786,NORDLAND GP.,NaN,19.468800,NaN,1.626459,...,34.779556,NaN,NaN,-0.574245,NaN,NaN,NaN,NaN,65000,1.0
3,15-09-2013,494.984,437641.9688,6470972.5,-469.957794,NORDLAND GP.,NaN,19.459282,NaN,1.621594,...,39.965164,NaN,NaN,-0.586315,NaN,NaN,NaN,NaN,65000,1.0
4,15-09-2013,495.136,437641.9688,6470972.5,-470.109772,NORDLAND GP.,NaN,19.453100,NaN,1.602679,...,57.483765,NaN,NaN,-0.597914,NaN,NaN,NaN,NaN,65000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114074,34/8-7 R,5436.024,476401.5000,6798676.0,-5394.967285,HEGRE GP.,Lunde Fm.,7.803589,9.195020,17.146124,...,NaN,NaN,NaN,0.052983,NaN,NaN,NaN,NaN,65000,2.0
114075,34/8-7 R,5436.176,476401.5313,6798676.0,-5395.116211,HEGRE GP.,Lunde Fm.,7.794892,10.846608,19.505636,...,NaN,NaN,NaN,0.047938,NaN,NaN,NaN,NaN,65000,2.0
114076,34/8-7 R,5436.328,476401.5625,6798676.0,-5395.265625,HEGRE GP.,Lunde Fm.,7.793001,11.811389,21.765100,...,NaN,NaN,NaN,0.044103,NaN,NaN,NaN,NaN,65000,2.0
114077,34/8-7 R,5436.480,476401.5938,6798676.0,-5395.414551,HEGRE GP.,Lunde Fm.,7.793001,11.721393,22.993740,...,NaN,NaN,NaN,0.042677,NaN,NaN,NaN,NaN,65000,2.0


In [6]:
lithology_numbers = {30000: 'Sandstone',
                     65030: 'Sandstone/Shale',
                     65000: 'Shale',
                     80000: 'Marl',
                     74000: 'Dolomite',
                     70000: 'Limestone',
                     70032: 'Chalk',
                     88000: 'Halite',
                     86000: 'Anhydrite',
                     99000: 'Tuff',
                     90000: 'Coal',
                     93000: 'Basement'}

In [7]:
trainData['LITH'] = trainData['FORCE_2020_LITHOFACIES_LITHOLOGY'].map(lithology_numbers)

In [8]:
trainData

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,...,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE,LITH
0,15-09-2013,494.528,437641.9688,6470972.5,-469.501831,NORDLAND GP.,NaN,19.480835,NaN,1.611410,...,NaN,NaN,-0.574928,NaN,NaN,NaN,NaN,65000,1.0,Shale
1,15-09-2013,494.680,437641.9688,6470972.5,-469.653809,NORDLAND GP.,NaN,19.468800,NaN,1.618070,...,NaN,NaN,-0.570188,NaN,NaN,NaN,NaN,65000,1.0,Shale
2,15-09-2013,494.832,437641.9688,6470972.5,-469.805786,NORDLAND GP.,NaN,19.468800,NaN,1.626459,...,NaN,NaN,-0.574245,NaN,NaN,NaN,NaN,65000,1.0,Shale
3,15-09-2013,494.984,437641.9688,6470972.5,-469.957794,NORDLAND GP.,NaN,19.459282,NaN,1.621594,...,NaN,NaN,-0.586315,NaN,NaN,NaN,NaN,65000,1.0,Shale
4,15-09-2013,495.136,437641.9688,6470972.5,-470.109772,NORDLAND GP.,NaN,19.453100,NaN,1.602679,...,NaN,NaN,-0.597914,NaN,NaN,NaN,NaN,65000,1.0,Shale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114074,34/8-7 R,5436.024,476401.5000,6798676.0,-5394.967285,HEGRE GP.,Lunde Fm.,7.803589,9.195020,17.146124,...,NaN,NaN,0.052983,NaN,NaN,NaN,NaN,65000,2.0,Shale
114075,34/8-7 R,5436.176,476401.5313,6798676.0,-5395.116211,HEGRE GP.,Lunde Fm.,7.794892,10.846608,19.505636,...,NaN,NaN,0.047938,NaN,NaN,NaN,NaN,65000,2.0,Shale
114076,34/8-7 R,5436.328,476401.5625,6798676.0,-5395.265625,HEGRE GP.,Lunde Fm.,7.793001,11.811389,21.765100,...,NaN,NaN,0.044103,NaN,NaN,NaN,NaN,65000,2.0,Shale
114077,34/8-7 R,5436.480,476401.5938,6798676.0,-5395.414551,HEGRE GP.,Lunde Fm.,7.793001,11.721393,22.993740,...,NaN,NaN,0.042677,NaN,NaN,NaN,NaN,65000,2.0,Shale
